In [9]:
from Enviroment import Enviroment
import pygame
import time
import random
import numpy as np
import os 

import matplotlib.pyplot as plt

import gym
from gym import spaces
import imageio

In [2]:
class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, obstacle_turn = False, vizualaze = False, Total_war = True, head_velocity = 0.01,steps_limit = 2000):
        super(CustomEnv, self).__init__()


        self.steps_limit = steps_limit
        
        self.env1 = Enviroment(obstacle_turn, vizualaze, Total_war, head_velocity)
        state = self.env1.reset()

        self.action_space = spaces.Discrete(8)
        
        self.observation_space = gym.spaces.Dict({
                    'img': spaces.Box(low=0, high=255, shape=(500, 500, 3), dtype=np.uint8),
                    'posRobot': spaces.Box(low=np.array([0, 0,-3.14]), high=np.array([500, 500, 3.14])),
                    'target': spaces.Box(low=np.array([0, 0,-3.14]), high=np.array([500, 500, 3.14]))
                    })
        self.log_koef = 50
        
    def step(self, action):

        state, reward, done, numstep = self.env1.step(action)
        
        dict_state = {'img':     state.img,
                      'posRobot':state.posRobot,
                      'target':  state.target}
        
#         dist = np.sqrt((dict_state['target'][0]-dict_state['posRobot'][0])**2 + (dict_state['target'][1]-dict_state['posRobot'][1])**2)
        
#         Ax = np.cos(dict_state['target'][2])
#         Ay = np.sin(dict_state['target'][2])
#         Bx = dict_state['target'][0] - dict_state['posRobot'][0]
#         By = dict_state['target'][1] - dict_state['posRobot'][1]

#         phy = np.arccos((Ax*Bx + Ay*By)/(np.sqrt(Ax**2 + Ay**2) * np.sqrt(Bx**2 + By**2)))
        
#         reward = reward + np.log2(phy/dist*self.log_koef )
        
        if numstep >= self.steps_limit:
            done = True
    
        return dict_state, reward, done, {}


    def reset(self):

        state = self.env1.reset()
        
        dict_state = {'img':     state.img,  
                      'posRobot':state.posRobot,  
                      'target':  state.target}  
        
        return dict_state  

    def render(self, model, num_gifs=1):
            for i in range(num_gifs):
                images = []
                obs = self.reset()
                img = obs['img']# 
                done = False
                while not done:
                    images.append(img)
                    action, _ = model.predict(obs)
                    obs, _, done ,_ = self.step(action)
                    img = obs['img'] #env.render(mode='rgb_array')

                imageio.mimsave(f"video{i}.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=30)

In [3]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch
import gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 2310):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        
        extractors = {}
        
        for key, subspace in observation_space.spaces.items():
            if key == "img":
        
                n_input_channels = observation_space[key].shape[0]
            
                extractors[key] = nn.Sequential(

                nn.Conv2d(n_input_channels, 32, 2),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(32, 64, 2),
                nn.MaxPool2d(2, 2),

                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(4, 4),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2), 
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(64, 128, 2),
                nn.Flatten())
                    
            elif key == "posRobot":
                            
                extractors[key] = nn.Sequential(nn.Linear(3, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                    
            elif key == "target":
                            
                extractors[key] = nn.Sequential(nn.Linear(3, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                
        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations: th.Tensor) -> th.Tensor:
        encoded_tensor_list = []

        # self.extractors contain nn.Modules that do all the processing.
        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))
        # Return a (B, self._features_dim) PyTorch tensor, where B is batch dimension.
        return th.cat(encoded_tensor_list, dim=1)


    
class ResBlock(nn.Module):
    def __init__(self, n_filters, kernel_size):
        super().__init__()
        self.n_filters = n_filters # the number of filters represents the number of output channels AND the number of input channels
        self.kernel_size = kernel_size

        self.b1 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
        
        self.b2 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99) # in keras the default epsilon = 0.001 & momentum = 0.99
        # whereas in putorch the default epsilon = 1e-05 & momentom = 0.1, the output is the same shape of input (N,C,H,W).
        self.b3 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
        self.b4 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        
    def forward(self, x):
        residual = x
        y = F.relu(self.b1(x))
        y = self.b2(y)
        y = F.relu(self.b3(y))
        y = self.b4(y)
        y += residual
        y = F.relu(y)
        return y

/home/user/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [4]:
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback

class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, agent_name: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model_' + agent_name)
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print("Num timesteps: {}".format(self.num_timesteps))
                print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print("Saving new best model to {}".format(self.save_path))
                  self.model.save(self.save_path)

        return True

In [5]:
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim = 518)
)

In [6]:
env = CustomEnv(obstacle_turn = False, 
                vizualaze     = False, 
                Total_war     = True,
                head_velocity = 0.07, 
                steps_limit   = 2000)

/home/user/anaconda3/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [10]:
from stable_baselines3 import DQN  # 

log_dir = './saved_models_disc_mult/DQN/'
os.makedirs(log_dir, exist_ok=True)

callback = SaveOnBestTrainingRewardCallback(check_freq  = 1000, 
                                            log_dir     = log_dir,
                                            agent_name  = 'DQN')

env = Monitor(env, log_dir)

model = DQN(policy                  = 'MlpPolicy',
            env                     = env,
            learning_rate           = 0.0001,
            buffer_size             = 10000,
            batch_size              = 20,
            gamma                   = 0.99,
            tensorboard_log         = "./tensorboard_logs_disc_mult/",
            policy_kwargs           = policy_kwargs,
            verbose                 = 1,
            device                  = 'cuda')

Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=1e6,callback=callback)

Logging to ./tensorboard_logs_disc_mult/DQN_1
Num timesteps: 2000
Best mean reward: -inf - Last mean reward per episode: 59.55
Saving new best model to ./saved_models_disc_mult/DQN/best_model_DQN
Num timesteps: 3000
Best mean reward: 59.55 - Last mean reward per episode: -151.61
Num timesteps: 4000
Best mean reward: 59.55 - Last mean reward per episode: -151.61
Num timesteps: 5000
Best mean reward: 59.55 - Last mean reward per episode: -131.91
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.43e+03 |
|    ep_rew_mean      | -289     |
|    exploration rate | 0.946    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 36       |
|    time_elapsed     | 156      |
|    total timesteps  | 5717     |
----------------------------------
Num timesteps: 6000
Best mean reward: 59.55 - Last mean reward per episode: -289.23
Num timesteps: 7000
Best mean reward: 59.55 - Last mean reward per episode: -289.23

/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/conv.py:439: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv2d(input, weight, bias, self.stride,


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.26e+03 |
|    ep_rew_mean      | -185     |
|    exploration rate | 0.521    |
| time/               |          |
|    episodes         | 40       |
|    fps              | 36       |
|    time_elapsed     | 1373     |
|    total timesteps  | 50370    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.63     |
|    n_updates        | 92       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.15e+03 |
|    ep_rew_mean      | -182     |
|    exploration rate | 0.519    |
| time/               |          |
|    episodes         | 44       |
|    fps              | 36       |
|    time_elapsed     | 1388     |
|    total timesteps  | 50672    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.41     |
|    n_updates      

## Записать видео

In [ ]:
# env.render(model,1)

## tensorboard

In [ ]:
# !tensorboard --logdir ./tensorboard_logs_disc_mult/

## load model

In [ ]:
# model = PPO.load("./saved_models_disc_mult/TD3/best_model_PPo1", env=env)